In [15]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Datasets de Scikit-learn
from sklearn.datasets import load_breast_cancer, load_iris

# Preprocesamiento y selección de modelos
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler

# Modelos y Ensembles
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier

# Ajuste de hiperparámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Ignorar advertencias para una salida más limpia
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


### Ejericio 1

In [16]:
# Cargar el dataset
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# Es una buena práctica escalar los datos, aunque para modelos basados en árboles no es estrictamente necesario.
# Lo haremos para mantener consistencia.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Forma de los datos (X): {X.shape}")
print(f"Número de etiquetas (y): {len(y)}")
print(f"Clases: {cancer.target_names}")

Forma de los datos (X): (569, 30)
Número de etiquetas (y): 569
Clases: ['malignant' 'benign']


In [17]:
# Configuración de la validación cruzada
seed = 42
kfold = KFold(n_splits=10, random_state=seed, shuffle=True)

# Definir los modelos
# 1. BaggingClassifier con un árbol de decisión como estimador base
dt = DecisionTreeClassifier()
# Nota: 'estimator' reemplazó a 'base_estimator' en scikit-learn >= 1.2
bagging = BaggingClassifier(estimator=dt, n_estimators=100, random_state=seed)

# 2. RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=seed)

# 3. ExtraTreesClassifier
extra_trees = ExtraTreesClassifier(n_estimators=100, random_state=seed)

# Lista de modelos para iterar
models = [
    ('BaggingClassifier', bagging),
    ('RandomForestClassifier', random_forest),
    ('ExtraTreesClassifier', extra_trees)
]

# Entrenar y evaluar cada modelo
results = []
names = []

print("="*60)
print("EVALUACIÓN DE MODELOS DE BAGGING")
print("="*60)

for name, model in models:
    cv_results = cross_val_score(model, X_scaled, y, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print(f"{name}:")
    print(f"  Media: {cv_results.mean():.4f}")
    print(f"  Desviación estándar: {cv_results.std():.4f}")
    print(f"  Rango: [{cv_results.min():.4f}, {cv_results.max():.4f}]")
    print()

print("✓ Evaluación completada.")

EVALUACIÓN DE MODELOS DE BAGGING
BaggingClassifier:
  Media: 0.9596
  Desviación estándar: 0.0209
  Rango: [0.9123, 0.9825]

BaggingClassifier:
  Media: 0.9596
  Desviación estándar: 0.0209
  Rango: [0.9123, 0.9825]

RandomForestClassifier:
  Media: 0.9631
  Desviación estándar: 0.0094
  Rango: [0.9474, 0.9825]

RandomForestClassifier:
  Media: 0.9631
  Desviación estándar: 0.0094
  Rango: [0.9474, 0.9825]

ExtraTreesClassifier:
  Media: 0.9718
  Desviación estándar: 0.0229
  Rango: [0.9107, 1.0000]

✓ Evaluación completada.
ExtraTreesClassifier:
  Media: 0.9718
  Desviación estándar: 0.0229
  Rango: [0.9107, 1.0000]

✓ Evaluación completada.


### Ejercicio 2


In [7]:
# Cargar el dataset
iris = load_iris()
X_iris, y_iris = iris.data, iris.target

# Escalar los datos es importante para KNN y Redes Neuronales
scaler_iris = StandardScaler()
X_iris_scaled = scaler_iris.fit_transform(X_iris)

print(f"Forma de los datos (X_iris): {X_iris.shape}")
print(f"Número de etiquetas (y_iris): {len(y_iris)}")
print(f"Clases: {iris.target_names}")

Forma de los datos (X_iris): (150, 4)
Número de etiquetas (y_iris): 150
Clases: ['setosa' 'versicolor' 'virginica']


In [8]:
# 1. Árbol de Decisión
dt_iris = DecisionTreeClassifier(random_state=seed)

# 2. K-Nearest Neighbors (KNN)
knn_iris = KNeighborsClassifier()

# 3. Red Neuronal (Multi-layer Perceptron)
# Usaremos una capa oculta con 10 neuronas como ejemplo de "simple capa".
mlp_iris = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=seed)

In [9]:
# Crear un ensemble de Boosting (AdaBoost) con árboles de decisión
# AdaBoost funciona mejor con aprendices débiles, por eso usamos max_depth=1
base_estimator_boost = DecisionTreeClassifier(max_depth=1)
# Nota: 'estimator' reemplazó a 'base_estimator' en scikit-learn >= 1.2
adaboost_ensemble = AdaBoostClassifier(estimator=base_estimator_boost, n_estimators=50, random_state=seed)

# Evaluar el rendimiento
kfold_iris = KFold(n_splits=10, random_state=seed, shuffle=True)
ada_results = cross_val_score(adaboost_ensemble, X_iris_scaled, y_iris, cv=kfold_iris, scoring='accuracy')

print("="*60)
print("ENSEMBLE ADABOOST - RESULTADOS")
print("="*60)
print(f"Rendimiento del Ensemble AdaBoost:")
print(f"  Media: {ada_results.mean():.4f}")
print(f"  Desviación estándar: {ada_results.std():.4f}")
print(f"  Rango: [{ada_results.min():.4f}, {ada_results.max():.4f}]")
print("\n✓ Evaluación completada.")

ENSEMBLE ADABOOST - RESULTADOS
Rendimiento del Ensemble AdaBoost:
  Media: 0.9467
  Desviación estándar: 0.0499
  Rango: [0.8667, 1.0000]

✓ Evaluación completada.


In [10]:
# Crear una lista de tuplas (nombre, modelo) para el VotingClassifier
estimators = [
    ('decision_tree', dt_iris),
    ('knn', knn_iris),
    ('mlp', mlp_iris)
]

# Crear el ensemble de votación
# 'hard' voting usa la clase predicha por la mayoría de los modelos.
voting_ensemble = VotingClassifier(estimators=estimators, voting='hard')

# Evaluar el rendimiento
voting_results = cross_val_score(voting_ensemble, X_iris_scaled, y_iris, cv=kfold_iris, scoring='accuracy')

print(f"Rendimiento del Ensemble de Votación: {voting_results.mean():.4f} ({voting_results.std():.4f})")

Rendimiento del Ensemble de Votación: 0.9600 (0.0533)


In [11]:
print("--- Ajustando Árbol de Decisión ---")
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_dt = GridSearchCV(estimator=DecisionTreeClassifier(random_state=seed), 
                              param_grid=param_grid_dt, 
                              cv=kfold_iris, 
                              scoring='accuracy')
grid_search_dt.fit(X_iris_scaled, y_iris)

print(f"Mejor Puntuación (Accuracy): {grid_search_dt.best_score_:.4f}")
print("Mejores Parámetros:")
print(grid_search_dt.best_params_)

--- Ajustando Árbol de Decisión ---
Mejor Puntuación (Accuracy): 0.9667
Mejores Parámetros:
{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
Mejor Puntuación (Accuracy): 0.9667
Mejores Parámetros:
{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}


In [12]:
print("\n--- Ajustando K-Nearest Neighbors (KNN) ---")
param_grid_knn = {
    'n_neighbors': list(range(1, 16)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

grid_search_knn = GridSearchCV(estimator=KNeighborsClassifier(), 
                               param_grid=param_grid_knn, 
                               cv=kfold_iris, 
                               scoring='accuracy')
grid_search_knn.fit(X_iris_scaled, y_iris)

print(f"Mejor Puntuación (Accuracy): {grid_search_knn.best_score_:.4f}")
print("Mejores Parámetros:")
print(grid_search_knn.best_params_)


--- Ajustando K-Nearest Neighbors (KNN) ---
Mejor Puntuación (Accuracy): 0.9667
Mejores Parámetros:
{'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}
Mejor Puntuación (Accuracy): 0.9667
Mejores Parámetros:
{'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}


In [14]:
print("\n--- Ajustando Red Neuronal (MLP) ---")
param_dist_mlp = {
    'hidden_layer_sizes': [(10,), (50,), (10, 10), (50, 25)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}

random_search_mlp = RandomizedSearchCV(estimator=MLPClassifier(max_iter=2000, random_state=seed),
                                       param_distributions=param_dist_mlp,
                                       n_iter=50,  # Número de combinaciones a probar
                                       cv=kfold_iris,
                                       scoring='accuracy',
                                       random_state=seed)
random_search_mlp.fit(X_iris_scaled, y_iris)

print(f"Mejor Puntuación (Accuracy): {random_search_mlp.best_score_:.4f}")
print("Mejores Parámetros:")
print(random_search_mlp.best_params_)


--- Ajustando Red Neuronal (MLP) ---
Mejor Puntuación (Accuracy): 0.9733
Mejores Parámetros:
{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (10, 10), 'alpha': 0.0001, 'activation': 'tanh'}
Mejor Puntuación (Accuracy): 0.9733
Mejores Parámetros:
{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (10, 10), 'alpha': 0.0001, 'activation': 'tanh'}
